### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter

import math
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=0.95, criterion='gini', max_features="sqrt"):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.impFunc = self.__ginimp
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.impFunc = self.__entimp
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.impFunc = self.__misimp
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __ginimp(self, y):
        c = Counter(y)
        num_class_samples = np.array([c[i] for i in list(c)])
        num_samples = y.shape[0]
        return 1 - np.sum((num_class_samples / num_samples)**2)

    def __entimp(self, y):
        c = Counter(y)
        num_class_samples = np.array([c[i] for i in list(c)])
        num_samples = y.shape[0]

        return -np.sum((num_class_samples / num_samples) *
                       np.log2(num_class_class / num_samples))

    def __misimp(self, y):
        c = Counter(y)
        num_samples = y.shape[0]
        num_class_samples = np.array([c[i] for i in list(c)])

        return 1 - np.max(num_class_samples / num_samples)

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return (1 - ((np.array((l_c / l_s) ** 2).sum(axis=1)) *
                     ((l_s / (l_s + r_s)).T) +
                     (np.array((r_c / r_s) ** 2).sum(axis=1)) *
                     ((r_s / (l_s + r_s)).T)).T)

    def __entropy(self, l_c, l_s, r_c, r_s):
        # Ваш код в 1 строчку
        return -np.array((l_c / l_s) * np.log(l_c / l_s + 1e-8)).sum(axis=1) -\
                np.array(((r_c / r_s) * np.log(r_c / r_s + 1e-8))).sum(axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        # Ваш код в 1 строчку
        return 1 - (l_c / l_s + r_c / r_s).max(axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код в 1 строчку
        return feature_ids[range(max(1, int(np.sqrt(n_feature))))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        # Ваш код в 1 строчку
        return feature_ids[range(max(1, int(np.log2(n_feature))))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(max(1, int(n_feature)))  # Ваш код в 1 строчку

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):

        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):

        node = []
        num_samples = y.shape[0]
        if(((depth is not None) and (depth == self.max_depth)) or
           num_samples <= self.min_samples_split or
           (Counter(y).most_common(1)[0][1] / num_samples)
           >= self.sufficient_share):
            node.append(self.__class__.LEAF_TYPE)
            if(num_samples > 0):
                node.append(Counter(y).most_common(1)[0][0])

            self.tree[node_id] = node
            return self.impFunc(y)

        node.append(self.__class__.NON_LEAF_TYPE)
        features = self.get_feature_ids(x.shape[1])
        features_threshold = np.array(list(map(
            lambda i: list(self.__find_threshold(x[:, i], y)), features)))

        best_features = np.argmin(features_threshold[:][0])
        node.append(features[best_features])
        node.append(features_threshold[best_features][1])
        self.tree[node_id] = node
        x_left, x_rigth, y_left, y_rigth = self.__div_samples(x, y,
                                                              node[1],
                                                              node[2])

        if ((y_left.shape[0] > 0) and (y_rigth.shape[0] > 0)):
            left_node_i = self.__fit_node(x_left, y_left,
                                          2*node_id+1, depth+1)

            rigth_node_i = self.__fit_node(x_rigth, y_rigth,
                                           2*node_id+2, depth+1)

            tmp = self.impFunc(y) * (num_samples) -\
                left_node_i * (y_left.shape[0])

            self.feature_importances_[features[best_features]] += tmp -\
                rigth_node_i * (y_rigth.shape[0])
        else:
            node = []
            node.append(self.__class__.LEAF_TYPE)

            if(num_samples > 0):
                node.append(Counter(y).most_common(1)[0][0])
            self.tree[node_id] = node

        return self.impFunc(y)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        fi = self.feature_importances_
        self.feature_importances_ /=  np.sum(fi)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [11]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [12]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target,
                                                    test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [13]:
%time clf.fit(X_train, y_train)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.04 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
%time my_clf.fit(X_train, y_train)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 22.6 ms


## Проверка качества работы на wine

In [563]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8372183372183372

In [564]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.875

## Подготовка данных Speed Dating Data 

In [61]:
df = pd.read_csv('./data/Speed Dating Data Key.csv',encoding = 'latin1')

data = df[[ 'match','gender' ,'age_o', 'race_o','goal','race_o', 'round' ,
            'condtn',  'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 
            'pf_o_sha','dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
            'amb_o', 'shar_o',  'prob_o','met_o']].dropna().astype(np.int32)


y = np.array(data['match'])

X=np.array(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Проверка скорости работы на Speed Dating Data 

In [62]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [63]:
%time clf.fit(X_train, y_train)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 10.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [64]:
%time my_clf.fit(X_train, y_train)

CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 111 ms


## Проверка качества работы на Speed Dating Data

In [67]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.83426

In [75]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8317309172718428

## Задание 3

In [287]:
print(np.argsort(clf.feature_importances_)[-10:])
print(np.argsort(my_clf.feature_importances_)[-10:])

[ 8  7  6  5  4  3  2  1 20  0]
[12 11  3  4 10  8 19 14 17  0]


## Задание 4

In [278]:
grid = {
    'n_estimators': range(1, 10),
    'max_depth': range(1, 70)
}
clf = RandomForestClassifier()
searcher = GridSearchCV(estimator=clf, param_grid=grid,cv = 4)
searcher.fit(X_train, y_train)
print(searcher.best_params_)


{'max_depth': 6, 'n_estimators': 7}


## Задание 5